In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import display

from PIL import Image
from src.draw_util import draw_x_at, pos_to_lines, composite_at

In [ ]:
dartboard_filename = 'res/dart_board.png'

In [ ]:
dartboard_image = Image.open(dartboard_filename)
dartboard_image

In [ ]:
width, height = dartboard_image.size
print(width, height)
left = 450
top = 350
right = width - 480
bottom = height
cropped = dartboard_image.crop((left, top, right, bottom))
cropped.save('res/dart_board_cropped.png')
cropped

In [ ]:
cropped_with_anchors = cropped.copy()

top_anchor = (495, 195)
left_anchor = (232, 425)
right_anchor = (750, 442)
bottom_anchor = (487, 643)

cropped_with_anchors = draw_x_at(cropped_with_anchors, top_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, left_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, right_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, bottom_anchor)

display(cropped_with_anchors)

In [ ]:
# draw dart on background board
dart_1 = Image.open('res/darts/dart_1.png')
composite = composite_at(cropped, dart_1, (300, 200))
composite.convert('RGB')

In [ ]:
def dart_position_in_cropped(pasted_at_pos, dart_top_pos):
    return (pasted_at_pos[0] + dart_top_pos[0], pasted_at_pos[1] + dart_top_pos[1])

def dart_position_in_original(cropped_at_pos, pasted_at_pos, dart_top_pos):
    return (cropped_at_pos[0] + pasted_at_pos[0] + dart_top_pos[0], cropped_at_pos[1] + pasted_at_pos[1] + dart_top_pos[1])

In [ ]:
import json

darts_info = 'darts.json'
with open(darts_info) as f:
    darts = json.load(f)

for t in darts:
    print(f"dart id {t} with {len(darts[t])} images")
    info = darts[t]
    for i in info:
        i['img'] = Image.open(i['path'])
    darts[t] = info

In [ ]:
###
### generate data
###
import random
import json

import numpy as np

def random_position_normal(start_pos, end_pos,  mean=0, std_dev=0.7):
    # Calculate the center of the rectangle
    center = ((start_pos[0] + end_pos[0]) / 2, (start_pos[1] + end_pos[1]) / 2)

    # Generate random values from a normal distribution around the center
    random_values_x = np.random.normal(loc=center[0], scale=std_dev, size=1)
    random_values_y = np.random.normal(loc=center[1], scale=std_dev, size=1)

    # Clip the values to ensure they are within the rectangle
    random_values_x = np.clip(random_values_x, start_pos[0], end_pos[0])
    random_values_y = np.clip(random_values_y, start_pos[1], end_pos[1])
    
    return (int(random_values_x[0]), int(random_values_y[0]))

def random_position(start_pos, end_pos):
    x = random.randint(start_pos[0], end_pos[0])
    y = random.randint(start_pos[1], end_pos[1])
    return (x, y)

def random_positions(start_pos, end_pos, n=3):
    positions = []
    for _ in range(n):
        x = random.randint(start_pos[0], end_pos[0])
        y = random.randint(start_pos[1], end_pos[1])
        positions.append((x, y))
    return positions


def build_label(filename, annotations, series_id):
    out_annotations = []
    for bbox, label in annotations:
        out_annotations.append({
            'bbox': bbox,
            'label': label
        })
    
    return {
        'image_filename': filename,
        'series_id': series_id,
        'annotations': out_annotations
    }

save_location = 'dartnet_data'


num_images = 1000
start_pos = (20, 20)
end_pos = cropped.size[0] - 20, cropped.size[1] - 20


print(f'cropped size {cropped.size}')

anchor_positions = [left_anchor, top_anchor, right_anchor, bottom_anchor]
anchor_annotations = [(a, 'anchor') for a in anchor_positions]

dart_ids = list(darts.keys())

labels = []
for i in range(num_images):
    print(f'iteration: {i}')
    dart_id = random.choice(dart_ids)
    dart_infos = darts[dart_id]
    

    dart_idx = random.randint(0, len(dart_infos) - 1)
    dart_1 = dart_infos[dart_idx]['img']
    dart_1_top_pos = dart_infos[dart_idx]['pos']
    end_pos_dart = (end_pos[0] - dart_1.size[0], end_pos[1] - dart_1.size[1])
    img_pos_1 = random_position(start_pos, end_pos_dart)
    composite_1 = composite_at(cropped, dart_1, img_pos_1)
    
    dart_idx = random.randint(0, len(dart_infos) - 1)
    dart_2 = dart_infos[dart_idx]['img']
    dart_2_top_pos = dart_infos[dart_idx]['pos']
    end_pos_dart = (end_pos[0] - dart_2.size[0], end_pos[1] - dart_2.size[1])
    img_pos_2 = random_position(start_pos, end_pos_dart)
    composite_2 = composite_at(composite_1, dart_2, img_pos_2)

    dart_idx = random.randint(0, len(dart_infos) - 1)
    dart_3 = dart_infos[dart_idx]['img']
    dart_3_top_pos = dart_infos[dart_idx]['pos']
    end_pos_dart = (end_pos[0] - dart_3.size[0], end_pos[1] - dart_3.size[1])
    img_pos_3 = random_position(start_pos, end_pos_dart)
    composite_3 = composite_at(composite_2, dart_3, img_pos_3)

    
    dart_1_pos_in_cropped = dart_position_in_cropped(img_pos_1, dart_1_top_pos)
    dart_2_pos_in_cropped = dart_position_in_cropped(img_pos_2, dart_2_top_pos)
    dart_3_pos_in_cropped = dart_position_in_cropped(img_pos_3, dart_3_top_pos)

    print(f'image_1_pos: {img_pos_1}, image_2_pos: {img_pos_2}, image_3_pos: {img_pos_3}')
    
    # save images as pngs
    filename_1 = f'{save_location}/dart_{i}_1.png'
    filename_2 = f'{save_location}/dart_{i}_2.png'
    filename_3 = f'{save_location}/dart_{i}_3.png'
    
    
    composite_1.save(filename_1)
    composite_2.save(filename_2)
    composite_3.save(filename_3)
    
    combinations = [
        [filename_1, [(dart_1_pos_in_cropped, 'dart'), *anchor_annotations]],
        [filename_2, [(dart_1_pos_in_cropped, 'dart'), (dart_2_pos_in_cropped, 'dart'), *anchor_annotations]],
        [filename_3, [(dart_1_pos_in_cropped, 'dart'), (dart_2_pos_in_cropped, 'dart'), (dart_3_pos_in_cropped, 'dart'), *anchor_annotations]],
    ]
    l1 = build_label(*combinations[0], i)
    l2 = build_label(*combinations[1], i)
    l3 = build_label(*combinations[2], i)
    
    
    labels += [l1, l2, l3]

with open(f'{save_location}/labels.json', 'w') as json_file:
    json_str = json.dumps({'labels': labels}, indent=4)
    print(json_str)
    json_file.write(json_str)


l = labels[2]
img = Image.open(l['image_filename'])
for a in l['annotations']:
    bbox = a['bbox']
    print(f'draw bbox {bbox}')
    img = draw_x_at(img, (bbox[0], bbox[1]), length=10)

img.convert('RGB')